<a href="https://colab.research.google.com/github/Ali-Fartout/Kaggle/blob/main/TPS_Jan_2022/TPS_jan_2022_LinearModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requierment

In [ ]:
! mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c tabular-playground-series-jan-2022

In [ ]:
import zipfile

zipfile = zipfile.ZipFile('/content/tabular-playground-series-jan-2022.zip','r')
zipfile.extractall()
zipfile.close()

In [ ]:
import pandas as pd
import numpy as np

train = pd.read_csv('/content/train.csv',parse_dates=['date'])
test = pd.read_csv('/content/test.csv',parse_dates=['date'])
sub = pd.read_csv('/content/sample_submission.csv')

# Feature engineering

In [ ]:
def OneHotEncoder(data,cols):
    for col in cols:
        col_ohe = pd.get_dummies(data[col], prefix=col)
        data = pd.concat((data, col_ohe), axis=1).drop(col, axis=1)

    # data['year'] = data.date.dt.year
    data['day'] = data.date.dt.day
    data['month'] = data.date.dt.month
    data['saturday'] = (data.date.dt.weekday == 5).astype(int)
    data['sunday'] = (data.date.dt.weekday == 6).astype(int)
    # data['endofmonth'] = (data.date.dt.day < 19  ).astype(int)
    data['endofyear'] = ((data.date.dt.month == 12) & (data.date.dt.day > 23 )).astype(int) 
    return data


train = OneHotEncoder(train,['product','store','country'])
test = OneHotEncoder(test,['product','store','country'])

In [ ]:
y = train.pop('num_sold')
yTrain = pd.DataFrame(y,columns=['num_sold'])
xTrain = train.drop(columns=['date','row_id'])
xTest = test.drop(columns=['date','row_id'])

# Model

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn_reg = KNeighborsRegressor(n_neighbors=7)
knn_reg.fit(xTrain,yTrain)
pred = knn_reg.predict(xTest)

In [ ]:
sub = pd.DataFrame()
sub['row_id'] = test['row_id']
sub['num_sold'] = pred
sub.to_csv('KNN.csv',index=False )

In [ ]:
!kaggle competitions submit  -c tabular-playground-series-jan-2022 -f /content/KNN.csv -m "KNN"
!kaggle competitions submissions  -c tabular-playground-series-jan-2022

100% 146k/146k [00:03<00:00, 38.9kB/s]
Successfully submitted to Tabular Playground Series - Jan 2022fileName            date                 description     status    publicScore  privateScore  
------------------  -------------------  --------------  --------  -----------  ------------  
KNN.csv             2022-03-15 09:40:17  KNN             complete  9.52083      9.77204       
KNN.csv             2022-03-15 09:34:23  KNN             complete  9.52083      9.77204       
SVM.csv             2022-03-15 09:25:40  SVM             complete  47.34849     41.88867      
KNN.csv             2022-03-15 09:23:10  KNN             complete  9.52083      9.77204       
KNN.csv             2022-03-15 09:22:58  KNN             complete  9.83408      10.09789      
KNN.csv             2022-03-15 09:22:46  KNN             complete  9.65663      9.93692       
KNN.csv             2022-03-15 09:22:27  KNN             complete  9.33924      9.85648       
KNN.csv             2022-03-15 09:22:15  KNN

## SVM

In [ ]:
from sklearn import svm
svm_reg = svm.SVR()

from sklearn.model_selection import GridSearchCV
 
# defining parameter range
param_grid = {'C': [0.1, 10, 100], 'gamma': [0.1,0.01,0.001],'kernel': ['rbf','sigmoid']}
 
grid = GridSearchCV(svm_reg, param_grid, refit = True, verbose = 2)
 
# fitting the model for grid search
grid.fit(xTrain, yTrain.values.ravel())

print(grid.best_estimator,grid.best_params_)

In [ ]:
sub = pd.DataFrame()
sub['row_id'] = test['row_id']
sub['num_sold'] = pred
sub.to_csv('SVM.csv',index=False )

In [ ]:
!kaggle competitions submit  -c tabular-playground-series-jan-2022 -f /content/SVM.csv -m "SVM"
!kaggle competitions submissions  -c tabular-playground-series-jan-2022